In [3]:
import os, sys, json
src_path = os.path.abspath(os.path.join(".."))
if src_path not in sys.path:
    sys.path.append(src_path)

import matplotlib.pyplot as plt
from src.image_processor import default_image_transform
from src.config import CaptionConfig
from src.data_loader import CaptionDataManager

%matplotlib inline
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [5]:
config_path = ".config.yml"
print(os.getcwd())
print(src_path)

/Users/nsimsiri/Documents/code/ml/nlp/cap/notebooks
/Users/nsimsiri/Documents/code/ml/nlp/cap
